# Exploratory Data Analysis

Perform initial investigations to expose how the source data is formated.

In [ ]:
# Setup Notebook

import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv("../.env.development.local")

DB_CONFIG = {
    'host':os.environ.get("DB_HOST"),
    'user':os.environ.get("DB_USER"),
    'password':os.environ.get("DB_PASSWORD"),
}

# TODO Get from ENV file
SOURCE_DATA_FILE = "source/Sample - Superstore.csv"
# Load the Source CSV into a data frame
df_source = pd.read_csv(SOURCE_DATA_FILE)

In [137]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [138]:
df_source.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [139]:
df_source.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [140]:
# Question - Are Customer IDs associated more than one address
df_orders = df_source[['Customer ID','Country','Region', 'State','City','Postal Code']]
df_orders = df_orders.drop_duplicates()

# Now let's see if Order ID is repeated. If so that means an order can be sent to more than one address
df_orders['Customer ID'].is_unique

# Answer - Customer Ids associated with more than 1 address
# So the address columns cannot be a property of the Customer object without repeating customer ids

False

In [118]:
# Question - Are Order Ids ever sent to more than one address
df_orders = df_source[['Order ID','Country','Region', 'State','City','Postal Code']]
df_orders = df_orders.drop_duplicates()

# Now let's see if Order ID is repeated. If so that means an order can be sent to more than one address
df_orders['Order ID'].is_unique

# Answer - Order Ids are never sent to more than 1 address
# So the address columns can be a property of the Order object

True

In [141]:
# Question - Are Product Ids duplicated accross products
df_products = df_source[['Product ID', 'Category', 'Sub-Category','Product Name']]
df_products = df_products.drop_duplicates(keep='first')
display(df_products)
display(df_products.info())

,Product ID,Category,Sub-Category,Product Name
0,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase
1,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...
3,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table
4,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System
...,...,...,...,...
9521,TEC-PH-10002817,Technology,Phones,RCA ViSYS 25425RE1 Corded phone
9562,TEC-MA-10003589,Technology,Machines,Cisco 8961 IP Phone Charcoal
9604,OFF-AP-10003099,Office Supplies,Appliances,"Eureka Hand Vacuum, Bagless"
9673,TEC-PH-10002645,Technology,Phones,LG G2


<class 'pandas.core.frame.DataFrame'>
Index: 1894 entries, 0 to 9935
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product ID    1894 non-null   object
 1   Category      1894 non-null   object
 2   Sub-Category  1894 non-null   object
 3   Product Name  1894 non-null   object
dtypes: object(4)
memory usage: 74.0+ KB


None

In [142]:
# Now let's see if Order ID is repeated. If so that means an order can be sent to more than one address
df_products['Product ID'].is_unique

False

In [124]:
# We know we have duplicates for some Product IDs, find them
df_product_dups = df_products[df_products.duplicated('Product ID', keep=False)].sort_values('Product ID')
df_product_dups["appender"] = range(len(df_product_dups))
df_product_dups
# Answer - Product ID Does not see to be unique as an identifier for product columns
# So this data should be cleaned up

,Product ID,Category,Sub-Category,Product Name,appender
2471,FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish",0
2115,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases,1
66,FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray",2
128,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black",3
1459,FUR-FU-10001473,Furniture,Furnishings,DAX Wood Document Frame,4
...,...,...,...,...,...
1219,TEC-PH-10002200,Technology,Phones,Samsung Galaxy Note 2,59
2596,TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...,60
1378,TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone,61
922,TEC-PH-10004531,Technology,Phones,OtterBox Commuter Series Case - iPhone 5 & 5s,62


In [143]:
# clean - append ROW ID to records that have a Duplicate Product ID
# A duplicate Product ID, is when the ID is used in records with different Category, Sub-Category, Product Name

# Which records need to be updated?
# What to Update the record with?
for index, dup_row in df_product_dups.iterrows():

    # Get the matching row in the dups table
    pid = dup_row["Product ID"]
    cat = dup_row["Category"]
    sc = dup_row["Sub-Category"]
    pn = dup_row["Product Name"]

    # Update all the rows in original DF that match the duplicate criteria
    df_source.loc[ 
        (df_source["Product ID"]== pid) & 
        (df_source["Category"]== cat) & 
        (df_source["Sub-Category"]== sc) & 
        (df_source["Product Name"]== pn)
        ,"Product ID"] = pid + "-" + str(dup_row["appender"])

In [144]:

# Verify the desired records were updated
df_source.loc[df_product_dups.index]["Product ID"].sort_values()

2471     FUR-BO-10002213-0
2115     FUR-BO-10002213-1
66       FUR-CH-10001146-2
128      FUR-CH-10001146-3
1459     FUR-FU-10001473-4
               ...        
1219    TEC-PH-10002200-59
2596    TEC-PH-10002310-60
1378    TEC-PH-10002310-61
922     TEC-PH-10004531-62
2713    TEC-PH-10004531-63
Name: Product ID, Length: 64, dtype: object

In [145]:
# Verify that not all records were updated.
df_source["Product ID"]

0       FUR-BO-10001798
1       FUR-CH-10000454
2       OFF-LA-10000240
3       FUR-TA-10000577
4       OFF-ST-10000760
             ...       
9989    FUR-FU-10001889
9990    FUR-FU-10000747
9991    TEC-PH-10003645
9992    OFF-PA-10004041
9993    OFF-AP-10002684
Name: Product ID, Length: 9994, dtype: object

In [146]:
# Save the clean dataframe
import shutil

# Make a copy to preserve the original file, just in case
shutil.copy(SOURCE_DATA_FILE, SOURCE_DATA_FILE.replace('.csv', '-original.csv'))

# Save to CSV without index
df_source.to_csv(SOURCE_DATA_FILE, index=False)

In [147]:
# OFF-PA-10001970

df_source[df_source['Product ID'].str.contains("OFF-PA-10001970")]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
350,351,CA-2016-129714,9/1/2016,9/3/2016,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,...,10009,East,OFF-PA-10001970-34,Office Supplies,Paper,Xerox 1881,24.560,2,0.0,11.5432
352,353,CA-2016-129714,9/1/2016,9/3/2016,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,...,10009,East,OFF-PA-10001970-34,Office Supplies,Paper,Xerox 1881,49.120,4,0.0,23.0864
411,412,CA-2017-117457,12/8/2017,12/12/2017,Standard Class,KH-16510,Keith Herrera,Consumer,United States,San Francisco,...,94110,West,OFF-PA-10001970-35,Office Supplies,Paper,Xerox 1908,55.980,1,0.0,27.4302
437,438,CA-2016-147375,6/12/2016,6/14/2016,Second Class,PO-19180,Philisse Overcash,Home Office,United States,Chicago,...,60623,Central,OFF-PA-10001970-35,Office Supplies,Paper,Xerox 1908,313.488,7,0.2,113.6394
1071,1072,CA-2016-152814,4/28/2016,5/2/2016,Standard Class,EH-14005,Erica Hernandez,Home Office,United States,Denver,...,80219,West,OFF-PA-10001970-34,Office Supplies,Paper,Xerox 1881,29.472,3,0.2,9.9468
1117,1118,CA-2015-142755,9/4/2015,9/8/2015,Standard Class,CS-12355,Christine Sundaresam,Consumer,United States,Roswell,...,30076,South,OFF-PA-10001970-35,Office Supplies,Paper,Xerox 1908,279.900,5,0.0,137.1510
1191,1192,CA-2017-117212,2/26/2017,2/28/2017,Second Class,BT-11530,Bradley Talbott,Home Office,United States,Los Angeles,...,90036,West,OFF-PA-10001970-35,Office Supplies,Paper,Xerox 1908,223.920,4,0.0,109.7208
3663,3664,US-2017-129777,7/3/2017,7/9/2017,Standard Class,FM-14290,Frank Merwin,Home Office,United States,Quincy,...,2169,East,OFF-PA-10001970-34,Office Supplies,Paper,Xerox 1881,24.560,2,0.0,11.5432
3850,3851,CA-2015-142377,12/4/2015,12/9/2015,Standard Class,MS-17980,Michael Stewart,Corporate,United States,Springfield,...,65807,Central,OFF-PA-10001970-34,Office Supplies,Paper,Xerox 1881,85.960,7,0.0,40.4012
3949,3950,CA-2016-119963,11/18/2016,11/22/2016,Standard Class,SN-20710,Steve Nguyen,Home Office,United States,Pasadena,...,77506,Central,OFF-PA-10001970-34,Office Supplies,Paper,Xerox 1881,19.648,2,0.2,6.6312
